In [26]:
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [27]:
# Read CSV
city_data = "raw_data/city_data.csv"
ride_data = "raw_data/ride_data.csv"
city_df = pd.read_csv("raw_data/city_data.csv")
ride_df = pd.read_csv("raw_data/ride_data.csv")

# Outer merge the city and ride data on city
combined_data = pd.merge(city_df,ride_df,how='outer', on='city')
combined_data.head()

,city,driver_count,type,date,fare,ride_id
0,Kelseyland,63,Urban,2016-08-19 04:27:52,5.51,6246006544795
1,Kelseyland,63,Urban,2016-04-17 06:59:50,5.54,7466473222333
2,Kelseyland,63,Urban,2016-05-04 15:06:07,30.54,2140501382736
3,Kelseyland,63,Urban,2016-01-25 20:44:56,12.08,1896987891309
4,Kelseyland,63,Urban,2016-08-09 18:19:47,17.91,8784212854829


In [44]:
# Convert driver count to numeric
combined_data['driver_count']= pd.to_numeric(combined_data['driver_count'], errors='coerce') # size of bubble

# Group data by city
city_data = combined_data.groupby('city')
city_data

In [50]:
# Average ($) fare per city
fare = city_data['fare'].mean()
# Total number of rides per city
rides = city_data['ride_id'].size()
# Number of drivers per city
drivers = city_data['driver_count'] # size of bubble
# City type
city_type = city_data['type'] # color of bubble


In [51]:
# Create the bubble plot
city_data.plot(kind="scatter", x=fare, y=rides, s=drivers, grid=True, alpha=0.25, title="Pyber Ride Sharing Data (2016)")
city_data.plot

KeyError: '[ 23.92870968  20.60961538  37.31555556  23.625       21.98157895\n  25.10645161  33.71133333  36.606       25.39551724  31.05166667\n  34.59181818  22.97809524  21.4972      31.41615385  26.16909091\n  24.47821429  32.59947368  33.66090909  39.053       33.24428571\n  26.87666667  30.04375     25.47894737  32.31        23.03458333\n  32.00222222  21.4825      32.00666667  24.77935484  27.83166667\n  33.16555556  28.75277778  24.16947368  21.80642857  36.928       22.94703704\n  34.98        31.1675      29.79117647  27.3348      25.3496      26.61\n  24.65761905  22.22521739  28.42892857  49.62        30.49888889\n  26.84428571  43.5325      29.52047619  26.86181818  24.96678571\n  31.90285714  24.15772727  31.03461538  27.08428571  24.13        32.81058824\n  35.0425      28.45461538  28.4735      24.97181818  34.06913043\n  25.89961538  32.38666667  26.856       38.146       25.54933333\n  31.14461538  23.3075      26.31666667  28.242       31.8065625\n  25.88294118  31.1

In [24]:
# Create the bubble plot
import seaborn as sns
sns.set_style("ticks")
sizes = [10,20,30,40,50,60,70,80,90,100]
marker_size = pd.cut(city_data['driver_count]/2, [0,10,20,30,40,50,60,70,80,90,100], labels=sizes)
sns.lmplot(rides, fare, data=indicator2, hue='type', scatter_kws={'s':marker_size})

SyntaxError: EOL while scanning string literal (<ipython-input-24-a86109eb8509>, line 6)